## Accelerate Inference: Neural Network Pruning

In [44]:
!python --version

Python 3.7.17


In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow.keras.layers import *

print(tf.version.VERSION)

2.15.0


In [7]:
# untar
# !ls
# !tar -xvzf dataset.tar.gz
# load train
train_images = pickle.load(open('train_images.pkl', 'rb'))
train_labels = pickle.load(open('train_labels.pkl', 'rb'))
# load val
val_images = pickle.load(open('val_images.pkl', 'rb'))
val_labels = pickle.load(open('val_labels.pkl', 'rb'))

In [108]:
# Define the neural network architecture (don't change this)

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 25, 32)        896       
                                                                 
 activation (Activation)     (None, 25, 25, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 23, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 23, 23, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 11, 11, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 11, 11, 32)        0         
                                                        

In [ ]:
# you can use the default hyper-parameters for training,
# val accuracy ~72% after 50 epochs

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, weight_decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, batch_size=32, epochs=50,
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

Epoch 1/50
703/703 [==============================] - 13s 8ms/step - loss: 1.5175 - accuracy: 0.3058 - val_loss: 1.3404 - val_accuracy: 0.4289
Epoch 2/50
703/703 [==============================] - 4s 6ms/step - loss: 1.3376 - accuracy: 0.4274 - val_loss: 1.2781 - val_accuracy: 0.4622
Epoch 3/50
703/703 [==============================] - 5s 7ms/step - loss: 1.2780 - accuracy: 0.4624 - val_loss: 1.2151 - val_accuracy: 0.4855
Epoch 4/50
703/703 [==============================] - 5s 7ms/step - loss: 1.2284 - accuracy: 0.4885 - val_loss: 1.1716 - val_accuracy: 0.5133
Epoch 5/50
703/703 [==============================] - 4s 6ms/step - loss: 1.2040 - accuracy: 0.5032 - val_loss: 1.1579 - val_accuracy: 0.5188
Epoch 6/50
703/703 [==============================] - 5s 7ms/step - loss: 1.1672 - accuracy: 0.5201 - val_loss: 1.1112 - val_accuracy: 0.5497
Epoch 7/50
703/703 [==============================] - 4s 6ms/step - loss: 1.1423 - accuracy: 0.5341 - val_loss: 1.1118 - val_accuracy: 0.5366
Epoch

In [ ]:
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 13ms/step - loss: 0.7438 - accuracy: 0.7204


In [ ]:
model.save_weights('original_weights.h5')

In [160]:
def structured_pruning(layer, pruning_percent):
  """ For Conv2D layer """
  filters, biases = layer.get_weights()

  filters_reshaped = filters.reshape((-1, filters.shape[-1]))
  norms = np.linalg.norm(filters_reshaped, axis=0)

  threshold = np.percentile(norms, pruning_percent)

  mask = norms < threshold
  # print(mask)
  filters[:,:,:,mask == True] = 0
  # print(biases)
  biases[mask == True] = 0
  # print(biases)

  layer.set_weights([filters, biases])
  return layer

In [165]:
def neuron_pruning(layer, pruning_percent):
  """ For Dense layer (FC layer) """
  weights, biases =layer.get_weights()

  norms = np.linalg.norm(weights, axis=0)
  threshold = np.percentile(norms, pruning_percent)

  mask = norms < threshold
  weights[:, mask == True] = 0

  layer.set_weights([weights, biases])
  return layer

In [162]:
def prune_CNN(model, conv_percent, dense_percent):
  for i, layer in enumerate(model.layers):
    if isinstance(layer, Conv2D):
      model.layers[i] = structured_pruning(layer, conv_percent)
    elif isinstance(layer, Dense):
      model.layers[i] = neuron_pruning(layer, dense_percent)
  return model

In [193]:
def zero_weight_ratio(model):
    zero_weights_count = 0
    total_weights_count = 0
    for layer in model.layers:
        if isinstance(layer, Conv2D) or isinstance(layer, Dense):
            weights, _ = layer.get_weights()
            zero_weights_count += tf.reduce_sum(tf.cast(tf.equal(weights, 0), tf.int32)).numpy()
            total_weights_count += tf.size(weights).numpy()
    return zero_weights_count / total_weights_count

In [198]:
def calculate_model_score(model):
    accuracy = model.evaluate(val_images, val_labels, batch_size=128)[1]
    sparsity = zero_weight_ratio(model)

    if accuracy > 0.6 and sparsity > 0:
        score = (accuracy + sparsity) / 2
    else:
        score = 0

    print(f"Accuracy: {accuracy}")
    print(f"Sparsity: {sparsity}")
    print(f"Model Score: {score}")

    return score

In [200]:
# grid search
scores = []
configs = []
for conv_percent in [0, 1, 2, 3, 4]:
  for dense_percent in [10, 15, 20, 25, 30]:
    model.load_weights('original_weights.h5')
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, weight_decay=1e-6),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    model = prune_CNN(model, conv_percent, dense_percent)
    print(f' -- conv {conv_percent}% dense {dense_percent}% -- ')
    score = calculate_model_score(model)
    scores.append(score)
    configs.append((conv_percent, dense_percent))
print(max(scores))
print(configs[np.argmax(scores)])

 -- conv 0% dense 10% -- 
20/20 [==============================] - 2s 64ms/step - loss: 0.8993 - accuracy: 0.6463
Accuracy: 0.6463366150856018
Sparsity: 0.0907764629599611
Model Score: 0.36855653902278146
 -- conv 0% dense 15% -- 
20/20 [==============================] - 2s 66ms/step - loss: 0.8990 - accuracy: 0.6440
Accuracy: 0.6439604163169861
Sparsity: 0.13400335008375208
Model Score: 0.3889818832003691
 -- conv 0% dense 20% -- 
20/20 [==============================] - 2s 64ms/step - loss: 0.8962 - accuracy: 0.6436
Accuracy: 0.6435643434524536
Sparsity: 0.17895931269249474
Model Score: 0.4112618280724742
 -- conv 0% dense 25% -- 
20/20 [==============================] - 3s 116ms/step - loss: 0.8942 - accuracy: 0.6424
Accuracy: 0.6423762440681458
Sparsity: 0.22218619981628573
Model Score: 0.43228122194221574
 -- conv 0% dense 30% -- 
20/20 [==============================] - 2s 66ms/step - loss: 1.0462 - accuracy: 0.5232
Accuracy: 0.5231683254241943
Sparsity: 0.26800670016750416
Model

In [201]:
# best config: conv_percent=0%, dense_percent=20%
model.load_weights('original_weights.h5')
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, weight_decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
model = prune_CNN(model, 0, 25)
score = calculate_model_score(model)
model.save_weights("my_model_weights_neuron_pruning.h5")

20/20 [==============================] - 2s 66ms/step - loss: 0.8942 - accuracy: 0.6424
Accuracy: 0.6423762440681458
Sparsity: 0.22218619981628573
Model Score: 0.43228122194221574


In [ ]:
# you need to save the model's weights, naming it 'my_model_weights.h5'
# model.save_weights("my_model_weights.h5")

# running this cell will immediately download a file called 'my_model_weights.h5'
from google.colab import files
files.download("my_model_weights.h5")